In [1]:
from unsloth import FastLanguageModel

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [2]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Meta-Llama-3.1-8B-Instruct", max_seq_length=8192, dtype=None, 
    load_in_4bit= True)

==((====))==  Unsloth 2024.10.7: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA GeForce RTX 4070 SUPER. Max memory: 11.72 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.0+cu124. CUDA = 8.9. CUDA Toolkit = 12.4.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

<string>:209: SyntaxWarning: invalid escape sequence '\ '
<string>:210: SyntaxWarning: invalid escape sequence '\_'
<string>:211: SyntaxWarning: invalid escape sequence '\ '
<string>:209: SyntaxWarning: invalid escape sequence '\ '
<string>:210: SyntaxWarning: invalid escape sequence '\_'
<string>:211: SyntaxWarning: invalid escape sequence '\ '
Unsloth: We fixed a gradient accumulation bug, but it seems like you don't have the latest transformers version!
Please update transformers, TRL and unsloth via:
`pip install --upgrade --no-cache-dir unsloth git+https://github.com/huggingface/transformers.git git+https://github.com/huggingface/trl.git`


In [3]:
FastLanguageModel.for_inference(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaExtendedRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,),

In [4]:
from transformers import TextStreamer
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(tokenizer, chat_template="llama-3.1", 
                              mapping = {"role" : "from", 
                                         "content" : "value", 
                                         "user" : "human", 
                                         "assistant" : "gpt"},)

In [11]:
msgs = [
    {"from": "human", "value": "What would a raspberry say to a pear if it could talk?"}
]
template = tokenizer.apply_chat_template(msgs, tokenize=True, add_generation_prompt=True, 
                                       return_tensors="pt")
inputs = template.to("cuda")

In [14]:
template

tensor([[128000, 128006,   9125, 128007,    271,  38766,   1303,  33025,   2696,
             25,   6790,    220,   2366,     18,    198,  15724,   2696,     25,
            220,   1627,   5887,    220,   2366,     19,    271, 128009, 128006,
          26380, 128007,    271,   3923,   1053,    264,  94802,   2019,    311,
            264,  38790,    422,    433,   1436,   3137,     30, 128009, 128006,
          78191, 128007,    271]])

In [12]:
text_streamer = TextStreamer(tokenizer)

In [17]:
response = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 1024)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

<|eot_id|><|start_header_id|>human<|end_header_id|>

What would a raspberry say to a pear if it could talk?<|eot_id|><|start_header_id|>assistant<|end_header_id|>



That's a fun one. Here's a possible conversation:

Raspberry: "Hey there, Pear! You're looking lovely today. I must say, your smooth skin is quite the envy of us berries."

Pear: "Ah, thank you, Raspberry! You're looking quite tart and juicy yourself. I've always admired your vibrant color and the way you add a burst of flavor to desserts."

Raspberry: "Ha! Yes, we berries have a certain charm to us, don't we? But I must say, your sweetness is quite alluring. I've always wondered, what's it like being a pear?"

Pear: "Well, it's a bit more laid-back than being a raspberry, I suppose. I get to ripen slowly and enjoy the sunshine, whereas you berries are often picked quickly and turned into jams and preserves."

Raspberry: "I see what you mean. It's a trade-off, isn't it? But I think our roles are complementary. Without us berries, your sweetness might get lost in the mix. And without your sweetness, our tartness might be too overwhelming."

Pear: "Exactly! We're both essential in our ow

In [16]:
response

tensor([[128000, 128006,   9125, 128007,    271,  38766,   1303,  33025,   2696,
             25,   6790,    220,   2366,     18,    198,  15724,   2696,     25,
            220,   1627,   5887,    220,   2366,     19,    271, 128009, 128006,
          26380, 128007,    271,   3923,   1053,    264,  94802,   2019,    311,
            264,  38790,    422,    433,   1436,   3137,     30, 128009, 128006,
          78191, 128007,    271,   3923,    264,  50189,    323,  77361,   3488,
           2268,   2746,    264,  94802,   1436,   3137,     11,    433,   2643,
           2019,    311,    264,  38790,   1473,  55005,     11,  23910,     11,
            499,   2351,    779,  11113,    323,  27877,     13,    358,   2846,
            264,   2766,  45915,    323,  89800,    398,     11,    719,    358,
           1093,    311,   1781,    358,    923,    264,  21165,    315,  28361,
            311,    904,   6671,     13,   1226,   2351,   1093,    379,    258,
            323,  10587,    